## Computational Homework 9: Principal Components Analysis and Regression

In this assignment, we will work through an example of performing principal components analysis in Python, in particular focusing on the case when the data matrix $X$ has columns which are nearly linearly dependent. We will then introduce an application of PCA to regression, called principal components regression (PCR), which can be a useful approach the $X$ is approximately low-rank (i.e., in the presence of multicollinearity). 

### Part 1: PCA

For the sake of this assignment, we will use a simulated dataset which is generated in the following cell.

In [80]:
import numpy as np 
np.random.seed(42)

n, p = 100, 30
U = np.linalg.qr(np.random.normal(size=(n,p)))[0] 
D = np.diag([10*np.exp(-0.2*t) for t in range(1,p+1)]) 
X = U@D

**Problem 1.** (Prepping the data for PCA). Using the data matrix $X$ defined in the cell above, perform the following:

1. Construct the centered data matrix $X_c$, whose columns have mean zero (note, you should be able to use the centering matrix $H = I - \frac{1}{n}\mathbb{1}\mathbb{1}^\top$ to do this easily).
2. Using $X_c$ defined above, define compute the sample covariance matrix $C = \frac{1}{n}X_c^\top X_c$.

**Problem 2.** (Performing PCA). Now that you've computed the covariance matrix $C$, we can perform PCA by computing its eigenvalue decomposition. To do this using numpy, you can use the function `np.linalg.eigh` -- this will return two arrays, the first is a $p\times 1$ array containing the eigenvalues, and the second is a $p\times p$ array containing the eigenvectors. Make a scatterplot of the eigenvalues, sorted from largest to smallest (this usually called a "scree plot"). What do you notice about this scatterplot? What does it say about the dimensionality of the data? Is there any indication of colinearity in the columns of $X$?

**Problem 3.** (Evaluating PCA, choosing the number of components). Next, we will create two plots which will help us evaluate how many components we might want to keep for an analysis. To do this, we will a plot of the cumulative proportion of variance explained by the first $j$ components. To do this, compute 

$$
p_j = \frac{\sum_{k=1}^j \lambda_k}{\sum_{l=1}^p \lambda_l}
$$

where $\lambda_k$ is the $k^{th}$ eigenvalue (in sorted order, so $\lambda_1 > \lambda_2 > \cdots > \lambda_p$). Using this plot, determine the minimum number of components needed to explain $90\%$ of the variance in the original data.

### Part 2: Principal components regression

Previously in this class, we've discussed what can happen when we have some features which are either exactly or very close to being linear combinations of each other. When this happens, we sometimes say that there is _multicolinearity_ present in the data (though this term is usually loosely defined). We've also seen that in these situations, the usual least squares estimator $\hat{\beta} = (X^\top X)^{-1}X^\top y$ is unstable, because the matrix $(X^\top X)^{-1}$ either doesn't exist, or contains numerically very large values that result in inflated variances of the coefficients.

So far, we've discussed a few ways of dealing with this problem via regularization (i.e. Ridge regression and LASSO). Here, we show that principal components analysis offers another solution. For this, we will continue with the data matrix $X$ we defined above, and also create a vector of responses.

In [86]:
beta_star = np.random.normal(size=p)
y = np.dot(X, beta_star) + 0.5*np.random.normal(size=n)

**Problem 1.** (Performing PCR). Principal components analysis works as follows. Recall that given the eigenvalue decomposition $C = U\Lambda U^\top$ of the covariance matrix, we can comptue the $j^{th}$ principal component by projecting $X$ onto the $j^{th}$ eigenvector (i.e. the $j^{th}$ column of $U$):

$$
z_j = X u_j \in \mathbb{R}^n. 
$$

In principal components regression, we first select $k\leq p$ of the principal components and forming the $n\times k$ data matrix 

$$
Z_{k} = \begin{bmatrix} | & | & & |\\ z_1 & z_2 & \cdots & z_k\\ | & | & & |\end{bmatrix} \in \mathbb{R}^{n\times k}
$$

and performing least squares using this data matrix instead to get the coefficients 

$$
\hat{\beta}_{PCR}(k) = (Z_{k}^\top Z_{k})^{-1}Z_{k}^\top y \in \mathbb{R}^k. 
$$

Perform PCR using the value of $k$ determined in Problem 3 of Part 1 in this assignment (i.e. the $k$ which describes $90\%$ of the variance in the data).

**Problem 2.** (Selecting the number of components to keep). One way to select the number of components that we should use in PCR is by doing essentially what we just did in the previous part, i.e. plot the cumulative proportion of variance, and select a value of $k$ that seems to capture a sufficiently large proportion of the variance in the data. However, we can also use cross-validation to do this.

For $k \in \{1,2,3,\dots,30\}$, perform 5-fold cross validation with PCR and plot the average out-of-sample MSE from this as a function of $k$ (see Homework 6 if you need a refresher on cross validation). What value of $k$ appears best here? Referring back to the cumulative proportion of variance plot from Part 1, what proportion of variance in the data is accounted for at this value?

_Note:_ For this problem, you should compute the principal components using only the data in the training set for each fold, i.e. recompute an eigenvalue decomposition for the training data at each step.

**Problem 3.** (Comparing to Ridge regression). As we mentioned, an PCR can be considered as an alternative to Ridge regression when the data matrix $X$ is poorly conditioned. To compare the performance of PCR vs Ridge regression, perform the same 5-fold cross validation as in the previous problem, but instead fitting Ridge regression models across as range of $\lambda$ values. At the best value of $\lambda$, which model performs better, PCR or Ridge?